In [29]:
import torch
from torch import nn
from torchvision import datasets, transforms
from torch.utils.data import DataLoader

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(device)
train_data = datasets.MNIST(root='data', train=True, transform=transforms.ToTensor(), download=True)
test_data = datasets.MNIST(root='data', train=False, transform=transforms.ToTensor(), download=True)

train_loader = DataLoader(train_data, batch_size=64, shuffle=True)
test_loader = DataLoader(test_data, batch_size=64, shuffle=False)


cpu


In [30]:
#Here is the implementation of MPL from PyTorch doc.
#We will use this model as an application of automatic differentiation.
class MPL(nn.Module):
    def __init__(self):
        super().__init__()
        self.flatten = nn.Flatten()
        self.linear_relu_stack = nn.Sequential(
            nn.Linear(28*28, 512),
            nn.ReLU(),
            nn.Linear(512, 512),
            nn.ReLU(),
            nn.Linear(512, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            nn.Linear(10, 10),
            nn.ReLU(),
            
        )

    def forward(self, x):
        x = self.flatten(x)
        logits = self.linear_relu_stack(x)
        return logits

#Here is the implementation of MPL from PyTorch doc.
#We will use this model as an application of automatic differentiation.
    
model = MPL().to(device)
print(model)

MPL(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
    (5): ReLU()
    (6): Linear(in_features=10, out_features=10, bias=True)
    (7): ReLU()
    (8): Linear(in_features=10, out_features=10, bias=True)
    (9): ReLU()
    (10): Linear(in_features=10, out_features=10, bias=True)
    (11): ReLU()
    (12): Linear(in_features=10, out_features=10, bias=True)
    (13): ReLU()
    (14): Linear(in_features=10, out_features=10, bias=True)
    (15): ReLU()
  )
)


In [32]:
def train_loop(data_loader, model, optimizer, loss, device):
    model.train()
    for batch_idx, (data, target) in enumerate(data_loader):
        data, target = data.to(device), target.to(device)

        # Forward pass
        pred = model(data)
        loss_value = loss(pred, target)

        # Backward pass
        optimizer.zero_grad()
        loss_value.backward()
        optimizer.step()

        if batch_idx % 100 == 0:
            print(f"Train Epoch: {batch_idx} \tLoss: {loss_value.item():.6f}")
def test_loop(data_loader, model, loss, device):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in data_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += loss(output, target).item()
            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(data_loader.dataset)
    print(f"\nTest set: Average loss: {test_loss:.4f}, Accuracy: {correct}/{len(data_loader.dataset)} ({100. * correct / len(data_loader.dataset):.0f}%)\n")
    
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(1, 100 + 1):
    train_loop(train_loader, model, optimizer, loss_fn, device)
    test_loop(test_loader, model, loss_fn, device)



Train Epoch: 0 	Loss: 2.310971
Train Epoch: 100 	Loss: 1.626524
Train Epoch: 200 	Loss: 0.899854
Train Epoch: 300 	Loss: 0.861866
Train Epoch: 400 	Loss: 0.316421
Train Epoch: 500 	Loss: 0.447637
Train Epoch: 600 	Loss: 0.264396
Train Epoch: 700 	Loss: 0.271651
Train Epoch: 800 	Loss: 0.166694
Train Epoch: 900 	Loss: 0.303866

Test set: Average loss: 0.0041, Accuracy: 9301/10000 (93%)

Train Epoch: 0 	Loss: 0.263832
Train Epoch: 100 	Loss: 0.315245
Train Epoch: 200 	Loss: 0.258337
Train Epoch: 300 	Loss: 0.213248
Train Epoch: 400 	Loss: 0.063413
Train Epoch: 500 	Loss: 0.235138
Train Epoch: 600 	Loss: 0.150778
Train Epoch: 700 	Loss: 0.185732
Train Epoch: 800 	Loss: 0.142305
Train Epoch: 900 	Loss: 0.407327

Test set: Average loss: 0.0028, Accuracy: 9515/10000 (95%)

Train Epoch: 0 	Loss: 0.197053
Train Epoch: 100 	Loss: 0.163021
Train Epoch: 200 	Loss: 0.106046
Train Epoch: 300 	Loss: 0.230157
Train Epoch: 400 	Loss: 0.026067
Train Epoch: 500 	Loss: 0.118372
Train Epoch: 600 	Loss: 0.